In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("RandomForest").getOrCreate()

23/12/27 09:34:37 WARN Utils: Your hostname, tkroza-IdeaPad-L340-15IRH-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.15.10 instead (on interface wlp7s0)
23/12/27 09:34:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/27 09:34:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
carros = spark.read.csv("/home/tkroza/PycharmProjects/ML_with_pyspark/data/Carros.csv", header=True, inferSchema=True, sep=";")
print(carros.count())
carros.show(5)

32
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [3]:
from pyspark.ml.regression import RandomForestRegressor

In [4]:
# Usando RFormula para separar o dataframe em conjunto de variaveis dependente e independentes
from pyspark.ml.feature import RFormula
Rformula = RFormula(formula="HP ~ Consumo + Cilindros + Cilindradas", featuresCol="independent_features", labelCol="dependent_variable")
carrosrf = Rformula.fit(carros).transform(carros)
carrosrf.select("independent_features", "dependent_variable").show(5, truncate=False)

+--------------------+------------------+
|independent_features|dependent_variable|
+--------------------+------------------+
|[21.0,6.0,160.0]    |110.0             |
|[21.0,6.0,160.0]    |110.0             |
|[228.0,4.0,108.0]   |93.0              |
|[214.0,6.0,258.0]   |110.0             |
|[187.0,8.0,360.0]   |175.0             |
+--------------------+------------------+
only showing top 5 rows



In [7]:
from pyspark.ml.feature import Normalizer
normalizador = Normalizer(inputCol="independent_features", outputCol="independentenorm", p=1.0)
carrosnorm = normalizador.transform(carrosrf)

In [9]:
carrosnorm.select("independent_features", "dependent_variable", "independentenorm").show(5, truncate=False)

+--------------------+------------------+-------------------------------------------------------------+
|independent_features|dependent_variable|independentenorm                                             |
+--------------------+------------------+-------------------------------------------------------------+
|[21.0,6.0,160.0]    |110.0             |[0.11229946524064172,0.03208556149732621,0.8556149732620321] |
|[21.0,6.0,160.0]    |110.0             |[0.11229946524064172,0.03208556149732621,0.8556149732620321] |
|[228.0,4.0,108.0]   |93.0              |[0.6705882352941176,0.011764705882352941,0.3176470588235294] |
|[214.0,6.0,258.0]   |110.0             |[0.4476987447698745,0.012552301255230125,0.5397489539748954] |
|[187.0,8.0,360.0]   |175.0             |[0.33693693693693694,0.014414414414414415,0.6486486486486487]|
+--------------------+------------------+-------------------------------------------------------------+
only showing top 5 rows



In [19]:
CarrosTreino, CarrosTeste = carrosnorm.randomSplit([0.8, 0.2], seed=1)
print(CarrosTeste.count())
print(CarrosTreino.count())

6
26


In [20]:
rf = RandomForestRegressor(featuresCol="independentenorm", labelCol="dependent_variable", maxDepth=10,
                           numTrees=500, seed=20)
modelo = rf.fit(CarrosTreino)

23/12/27 09:58:49 WARN DecisionTreeMetadata: DecisionTree reducing maxBins from 32 to 26 (= number of training instances)
23/12/27 09:58:50 WARN DAGScheduler: Broadcasting large task binary with size 1330.9 KiB
23/12/27 09:58:50 WARN DAGScheduler: Broadcasting large task binary with size 1374.6 KiB


In [21]:
previsao = modelo.transform(CarrosTeste)
previsao.select("dependent_variable", "prediction").show()

+------------------+----------+
|dependent_variable|prediction|
+------------------+----------+
|             215.0|   198.592|
|             180.0|   162.214|
|             180.0|   162.214|
|             123.0|   123.038|
|             175.0|  147.2435|
|             113.0|   118.184|
+------------------+----------+



In [22]:
# Avaliando Modelo
from pyspark.ml.evaluation import RegressionEvaluator
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="dependent_variable", metricName="rmse")
rmse = avaliar.evaluate(previsao)
print(rmse)

16.828570192037503
